### Ввод регулярного выражения

In [54]:
regex = '(a|b)*abb'

### 1. Построение НКА

In [55]:
from mermaid import Mermaid
operations = {'|','*','?'}

#### Добавление явного символа для конкатенации

In [56]:
refined_concat_regex = []
for i in range(len(regex)):
    refined_concat_regex.append(regex[i])
    if i + 1 < len(regex):
        if regex[i] not in {'|', '('} and regex[i + 1] not in {'|', '*', ')'}:
            refined_concat_regex.append('?')

refined_concat_regex = ''.join(refined_concat_regex)
print('Регулярное выражение с операцией конкатенации: ' + refined_concat_regex) 

Регулярное выражение с операцией конкатенации: (a|b)*?a?b?b


#### Запись в обратной нотации

In [57]:
priority = {'(':-1, '|':0, '?':1, '*':2}

reversed_regex = list()
stack = list()

for symbol in refined_concat_regex:
    if symbol in operations:
        while len(stack) > 0 and priority[stack[-1]] >= priority[symbol]:
            reversed_regex.append(stack.pop())
        stack.append(symbol)
    elif symbol == '(':
        stack.append(symbol)
    elif symbol == ')':
        while len(stack) > 0 and stack[-1] != '(':
            reversed_regex.append(stack.pop())
        stack.pop()
    else:
        reversed_regex.append(symbol)
        
for i in range(len(stack)):
    reversed_regex.append(stack.pop())
        
reversed_regex = ''.join(reversed_regex)
print('Регулярное выражение в обратной нотации: ' + reversed_regex)

Регулярное выражение в обратной нотации: ab|*a?b?b?


#### Формирование НКА

In [58]:
stack = list()
q_table = list()
q_counter = 0
start_state = -1
finish_state = -1

for symbol in reversed_regex:
    if symbol == '|':
        right_sub_nfa = stack.pop()
        left_sub_nfa = stack.pop()
        q_counter += 1
        q_table.append({"start_state":q_counter, "final_state": left_sub_nfa[0], "symbol":'ε'})
        q_table.append({"start_state":q_counter, "final_state": right_sub_nfa[0], "symbol":'ε'})
        q_table.append({"start_state":left_sub_nfa[1], "final_state": q_counter + 1, "symbol":'ε'})
        q_table.append({"start_state":right_sub_nfa[1], "final_state": q_counter + 1, "symbol":'ε'})  
        stack.append((q_counter, q_counter + 1))
        start_state = q_counter
        finish_state = q_counter + 1
        q_counter += 1
    elif symbol == '*':
        sub_nfa = stack.pop()
        q_counter += 1
        q_table.append({"start_state":sub_nfa[1], "final_state": sub_nfa[0], "symbol":'ε'})
        q_table.append({"start_state":sub_nfa[1], "final_state": q_counter + 1, "symbol":'ε'})
        q_table.append({"start_state":q_counter, "final_state": sub_nfa[0], "symbol":'ε'})
        q_table.append({"start_state":q_counter, "final_state": q_counter + 1, "symbol":'ε'})
        stack.append((q_counter, q_counter + 1))
        start_state = q_counter
        finish_state = q_counter + 1
        q_counter += 1
    elif symbol == '?':
        right_sub_nfa = stack.pop()
        left_sub_nfa = stack.pop()
        q_table.append({"start_state":left_sub_nfa[1], "final_state": right_sub_nfa[0], "symbol":'ε'})  
        stack.append((left_sub_nfa[0], right_sub_nfa[1]))   
        start_state = left_sub_nfa[0]
        finish_state = right_sub_nfa[1]
    else:
        q_counter += 1
        q_table.append({"start_state":q_counter, "final_state": q_counter + 1, "symbol":symbol})
        stack.append((q_counter, q_counter + 1))
        start_state = q_counter
        finish_state = q_counter + 1
        q_counter += 1

In [59]:
print('Недетерминированный конечный автомат')
print('\nНабор состояний:')
print(', '.join(list(map(lambda x: str(x), set([rule['start_state'] for rule in q_table] + [rule['final_state'] for rule in q_table])))))

print('\nСтартовое состояние: {}, Финальное состояние: {}'.format(start_state, finish_state))

q_table = sorted(q_table, key=lambda x: x['start_state'])
print('\nТаблица переходов:')
print('Текущее состояние | Целевое состояние | Символ')
for rule in q_table:
    print('----------------------------------------------')
    print('{:^18}|{:^19}|{:^7}'.format(rule['start_state'], rule['final_state'], rule['symbol']))

Недетерминированный конечный автомат

Набор состояний:
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14

Стартовое состояние: 7, Финальное состояние: 14

Таблица переходов:
Текущее состояние | Целевое состояние | Символ
----------------------------------------------
        1         |         2         |   a   
----------------------------------------------
        2         |         6         |   ε   
----------------------------------------------
        3         |         4         |   b   
----------------------------------------------
        4         |         6         |   ε   
----------------------------------------------
        5         |         1         |   ε   
----------------------------------------------
        5         |         3         |   ε   
----------------------------------------------
        6         |         5         |   ε   
----------------------------------------------
        6         |         8         |   ε   
-------------------------------

In [60]:
graph = str()
for rule in q_table.copy():
    if rule['start_state'] == finish_state:
        graph+='{}((({})))'.format(rule['start_state'], rule['start_state'])
    else:
        graph+='{}(({}))'.format(rule['start_state'], rule['start_state'])
    graph+=' -->|{}|'.format(rule['symbol'])
    if rule['final_state'] == finish_state:
        graph+='{}((({})))\n'.format(rule['final_state'], rule['final_state'])
    else:
        graph+='{}(({}))\n'.format(rule['final_state'], rule['final_state'])
graph+='style {} fill:lime\n'.format(start_state)
print('\nГраф полученного НКА:')
Mermaid("graph LR\n" + graph)


Граф полученного НКА:


### 2. Детерминизация (НКА -> ДКА)

In [61]:
def e_closure(Q):
    result = Q
    stack = list(Q)
    while len(stack) > 0:
        curr_state = stack.pop()
        for u in set([rule['final_state'] for rule in q_table if rule['symbol'] == 'ε' and rule['start_state'] == curr_state]):
            if u not in result:
                result.add(u)
                stack.append(u)
    return result

def move(Q, a):
    a_closure_states = set([rule['final_state'] for rule in q_table if rule['symbol'] == a and rule['start_state'] in Q])
    return a_closure_states

def set_name(Q):
    Q = list(Q)
    name = '-'.join(list(map(lambda x: str(x), Q)))
    return name

#### Детерминизация

In [62]:
symbols = set(rule['symbol'] for rule in q_table)
symbols.remove('ε')

d_q_table = []
marked_states = [e_closure(set([start_state]))]
queue = [set([start_state])]

while len(queue) > 0:
    curr_states = queue.pop()
    curr_states = curr_states | e_closure(curr_states)
    for symbol in symbols:
        new_states = move(curr_states, symbol)  
        if e_closure(new_states) not in marked_states:
            queue.append(e_closure(new_states))
            marked_states.append(new_states | e_closure(new_states))
        if len(new_states) > 0:
            d_q_table.append({'start_state':list(curr_states),'final_state':list(e_closure(new_states)), 'symbol':symbol})

#### Поиск начальных и терминальных состояний в новом КА

In [63]:
d_start_states = list(filter(lambda x: start_state in x, marked_states))
d_finish_states = list(filter(lambda x: finish_state in x, marked_states))

In [64]:
print('Детерминированный конечный автомат')
print('\nНабор состояний:')
print(', '.join([set_name(state) for state in marked_states.copy() if len(state)>0]))

print('\nСтартовое состояние: {}, Финальное состояние: {}'.format(', '.join([set_name(state) for state in d_start_states]), ', '.join([set_name(state) for state in d_finish_states])))

print('\nТаблица переходов:')
print('{:^30}|{:^30}|{:^7}'.format('Текущее состояние', 'Целевое состояние', 'Символ'))
for rule in d_q_table.copy():
    print('-'*69)
    print('{:^30}|{:^30}|{:^7}'.format(set_name(rule['start_state']), set_name(rule['final_state']), rule['symbol']))

Детерминированный конечный автомат

Набор состояний:
1-3-5-7-8-9, 1-2-3-5-6-8-9-10-11, 1-3-4-5-6-8-9, 1-3-4-5-6-8-9-12-13, 1-3-4-5-6-8-9-14

Стартовое состояние: 1-3-5-7-8-9, Финальное состояние: 1-3-4-5-6-8-9-14

Таблица переходов:
      Текущее состояние       |      Целевое состояние       |Символ 
---------------------------------------------------------------------
         1-3-5-7-8-9          |     1-2-3-5-6-8-9-10-11      |   a   
---------------------------------------------------------------------
         1-3-5-7-8-9          |        1-3-4-5-6-8-9         |   b   
---------------------------------------------------------------------
        1-3-4-5-6-8-9         |     1-2-3-5-6-8-9-10-11      |   a   
---------------------------------------------------------------------
        1-3-4-5-6-8-9         |        1-3-4-5-6-8-9         |   b   
---------------------------------------------------------------------
     1-2-3-5-6-8-9-10-11      |     1-2-3-5-6-8-9-10-11      |   a 

In [65]:
graph = str()
for rule in d_q_table.copy():
    if set(rule['start_state']) in d_finish_states:
        graph+='{}((({})))'.format(set_name(rule['start_state']), set_name(rule['start_state']))
    else:
        graph+='{}(({}))'.format(set_name(rule['start_state']), set_name(rule['start_state']))
    graph+=' -->|{}|'.format(rule['symbol'])
    if set(rule['final_state']) in d_finish_states:
        graph+='{}((({})))\n'.format(set_name(rule['final_state']), set_name(rule['final_state']))
    else:
        graph+='{}(({}))\n'.format(set_name(rule['final_state']), set_name(rule['final_state']))
for d_start_state in d_start_states:
    graph+='style {} fill:lime\n'.format(set_name(d_start_state))
    print('\nГраф полученного ДКА:')
Mermaid("graph LR\n" + graph)


Граф полученного ДКА:


### 3. Минимизация ДКА

#### Сопоставление новых состояний

In [66]:
d_q_states = dict()
new_d_q_table = list()

for i in range(len(marked_states)):
    if len(marked_states[i]) > 0:
        d_q_states[set_name(marked_states[i])] = i + 1

new_d_start_states = [d_q_states[set_name(state)] for state in d_start_states]
new_d_finish_states = [d_q_states[set_name(state)] for state in d_finish_states]

for rule in d_q_table:
    new_rule = dict()
    new_rule['start_state'] = d_q_states[set_name(rule['start_state'])]
    new_rule['final_state'] = d_q_states[set_name(rule['final_state'])]
    new_rule['symbol'] = rule['symbol']
    new_d_q_table.append(new_rule)

In [67]:
print('Детерминированный конечный автомат (с новой нумерацией состояний)')
print('\nНабор состояний:')
print(', '.join([str(state) for state in list(d_q_states.values())]))

print('\nСтартовое состояние: {}, Финальное состояние: {}'.format(', '.join([str(state) for state in new_d_start_states]), ', '.join([str(state) for state in new_d_finish_states])))

print('\nТаблица переходов:')
print('{:^30}|{:^30}|{:^7}'.format('Текущее состояние', 'Целевое состояние', 'Символ'))
for rule in new_d_q_table.copy():
    print('-'*69)
    print('{:^30}|{:^30}|{:^7}'.format(rule['start_state'], rule['final_state'], rule['symbol']))

Детерминированный конечный автомат (с новой нумерацией состояний)

Набор состояний:
1, 2, 3, 4, 5

Стартовое состояние: 1, Финальное состояние: 5

Таблица переходов:
      Текущее состояние       |      Целевое состояние       |Символ 
---------------------------------------------------------------------
              1               |              2               |   a   
---------------------------------------------------------------------
              1               |              3               |   b   
---------------------------------------------------------------------
              3               |              2               |   a   
---------------------------------------------------------------------
              3               |              3               |   b   
---------------------------------------------------------------------
              2               |              2               |   a   
----------------------------------------------------------------

In [68]:
graph = str()
for rule in new_d_q_table.copy():
    if rule['start_state'] in new_d_finish_states:
        graph+='{}((({})))'.format(rule['start_state'], rule['start_state'])
    else:
        graph+='{}(({}))'.format(rule['start_state'], rule['start_state'])
    graph+=' -->|{}|'.format(rule['symbol'])
    if rule['final_state'] in new_d_finish_states:
        graph+='{}((({})))\n'.format(rule['final_state'], rule['final_state'])
    else:
        graph+='{}(({}))\n'.format(rule['final_state'], rule['final_state'])
for d_start_state in new_d_start_states:
    graph+='style {} fill:lime\n'.format(d_start_state)
    print('\nГраф полученного ДКА (с новой нумерацией):')
Mermaid("graph LR\n" + graph)


Граф полученного ДКА (с новой нумерацией):


#### Минимизация автомата

In [69]:
def split(R, C, a):
    R1 = set([rule['start_state'] for rule in new_d_q_table if rule['symbol'] == a and rule['final_state'] in C])
    #print('!', R, R1, C, a)
    R1 = R & R1
    R2 = R-R1
    return (R1, R2)

In [70]:
Q = set(d_q_states.values())
S = list()
F = set(new_d_finish_states)
P = list()
P.append(Q)
P.append(Q-F)
for symbol in symbols:
    S.append((F, symbol))
    S.append((Q-F, symbol))
while len(S) > 0:
    C, a = S.pop()
    for R in P:
        R1, R2 = split(R, C, a)
        if len(R1) > 0 and len(R2) > 0:
            P.remove(R)
            P.append(R1)
            P.append(R2)
            for symbol in symbols:
                S.append((R1, symbol))
                S.append((R2, symbol))
for state in P.copy():
    for other_state in P.copy():
        if len(state) > 0:
            if state != other_state and state.issubset(other_state):
                P.remove(other_state)
                P.append(other_state-state)

In [71]:
min_states_eqv = dict()
min_start_states = list()
min_finish_states = list()

for state in list(d_q_states.values()):
    min_states_eqv[state] = [list(min_state) for min_state in P if state in min_state][0]
    if state in new_d_start_states:
        min_start_states.append(min_states_eqv[state])
    if state in new_d_finish_states:
        min_finish_states.append(min_states_eqv[state])


min_table = list()

for rule in new_d_q_table:
    min_rule = {'start_state': set_name(min_states_eqv[rule['start_state']]), 'final_state': set_name(min_states_eqv[rule['final_state']]), 'symbol': rule['symbol']}
    min_table.append(min_rule)
         
#просто доверься - это удаление дубоикатов правил перехода
min_table = [{'start_state':tup[0], 'final_state':tup[1], 'symbol':tup[2]} for tup in set([tuple(rule.values()) for rule in min_table])]



In [72]:
print('Минимальный детерминированный конечный автомат')
print('\nНабор состояний:')
print(', '.join(set([set_name(state) for state in P.copy() if len(state) > 0])))

print('\nСтартовое состояние: {}, Финальное состояние: {}'.format(', '.join(set([set_name(state) for state in min_start_states])), ', '.join(set([set_name(state) for state in min_finish_states]))))

print('\nТаблица переходов:')
print('{:^30}|{:^30}|{:^7}'.format('Текущее состояние', 'Целевое состояние', 'Символ'))
for rule in min_table.copy():
    print('-'*69)
    print('{:^30}|{:^30}|{:^7}'.format(rule['start_state'], rule['final_state'], rule['symbol']))

Минимальный детерминированный конечный автомат

Набор состояний:
2, 4, 5, 1-3

Стартовое состояние: 1-3, Финальное состояние: 5

Таблица переходов:
      Текущее состояние       |      Целевое состояние       |Символ 
---------------------------------------------------------------------
             1-3              |             1-3              |   b   
---------------------------------------------------------------------
              2               |              4               |   b   
---------------------------------------------------------------------
              4               |              5               |   b   
---------------------------------------------------------------------
             1-3              |              2               |   a   
---------------------------------------------------------------------
              2               |              2               |   a   
---------------------------------------------------------------------
            

In [73]:
graph = str()
for rule in min_table.copy():
    if rule['start_state'] in [set_name(fs) for fs in min_finish_states]:
        graph+='{}((({})))'.format(rule['start_state'], rule['start_state'])
    else:
        graph+='{}(({}))'.format(rule['start_state'], rule['start_state'])
    graph+=' -->|{}|'.format(rule['symbol'])
    if rule['final_state'] in [set_name(fs) for fs in min_finish_states]:
        graph+='{}((({})))\n'.format(rule['final_state'], rule['final_state'])
    else:
        graph+='{}(({}))\n'.format(rule['final_state'], rule['final_state'])
for min_start_state in min_start_states:
    graph+='style {} fill:lime\n'.format(set_name(min_start_state))
    print('\nГраф полученного минимального ДКА:')
Mermaid("graph LR\n" + graph)


Граф полученного минимального ДКА:


### 4. Моделирование минимального ДКА

#### Сопоставление новых состояний

In [74]:
new_min_states = dict()
new_min_table = list()
old_states = list(set([set_name(state) for state in P.copy() if len(state) > 0]))
for i in range(len(old_states)):
    new_min_states[old_states[i]] = i + 1

new_min_start_states = [new_min_states[set_name(state)] for state in min_start_states]
new_min_finish_states = [new_min_states[set_name(state)] for state in min_finish_states]

for rule in min_table:
    new_rule = dict()
    new_rule['start_state'] = new_min_states[rule['start_state']]
    new_rule['final_state'] = new_min_states[rule['final_state']]
    new_rule['symbol'] = rule['symbol']
    new_min_table.append(new_rule)

In [75]:
print('Минимальный детерминированный конечный автомат (с новой нумерацией состояний)')
print('\nНабор состояний:')
print(', '.join([str(state) for state in list(new_min_states.values())]))

print('\nСтартовое состояние: {}, Финальное состояние: {}'.format(', '.join([str(state) for state in new_min_start_states]), ', '.join(set([str(state) for state in new_min_finish_states]))))

print('\nТаблица переходов:')
print('{:^30}|{:^30}|{:^7}'.format('Текущее состояние', 'Целевое состояние', 'Символ'))
for rule in new_min_table.copy():
    print('-'*69)
    print('{:^30}|{:^30}|{:^7}'.format(rule['start_state'], rule['final_state'], rule['symbol']))

Минимальный детерминированный конечный автомат (с новой нумерацией состояний)

Набор состояний:
1, 2, 3, 4

Стартовое состояние: 4, Финальное состояние: 3

Таблица переходов:
      Текущее состояние       |      Целевое состояние       |Символ 
---------------------------------------------------------------------
              4               |              4               |   b   
---------------------------------------------------------------------
              1               |              2               |   b   
---------------------------------------------------------------------
              2               |              3               |   b   
---------------------------------------------------------------------
              4               |              1               |   a   
---------------------------------------------------------------------
              1               |              1               |   a   
-------------------------------------------------------

In [76]:
graph = str()
for rule in new_min_table.copy():
    if rule['start_state'] in new_min_finish_states:
        graph+='{}((({})))'.format(rule['start_state'], rule['start_state'])
    else:
        graph+='{}(({}))'.format(rule['start_state'], rule['start_state'])
    graph+=' -->|{}|'.format(rule['symbol'])
    if rule['final_state'] in new_min_finish_states:
        graph+='{}((({})))\n'.format(rule['final_state'], rule['final_state'])
    else:
        graph+='{}(({}))\n'.format(rule['final_state'], rule['final_state'])
for d_start_state in new_d_start_states:
    graph+='style {} fill:lime\n'.format(new_min_start_states[0])
print('\nГраф полученного ДКА (с новой нумерацией):')
Mermaid("graph LR\n" + graph)


Граф полученного ДКА (с новой нумерацией):


#### Моделирование полученного КА

In [89]:
input_string = 'ab'

In [90]:
model_table = dict()

for rule in new_min_table:
    if rule['start_state'] in list(model_table.keys()):
        model_table[rule['start_state']][rule['symbol']] = rule['final_state']
    else:
        model_table[rule['start_state']] = {rule['symbol']:rule['final_state']}

In [91]:
available_states = set(model_table.keys())
current_state = new_min_start_states[0]
isOk = True
for char in input_string:
    print('Текущее состояние: {}\nПолученный символ из введенной строки: {}'.format(current_state, char))
    if current_state in available_states and char in list(model_table[current_state].keys()):
        current_state = model_table[current_state][char]
        print('Переход в состояние {}'.format(current_state))
    else:
        isOk = False
        print('Нет правила перехода из {} по {}'.format(current_state, char))
        break
if current_state not in new_min_finish_states:
    print('Состояние автомата не конечное')
    isOk = False

if isOk:
    print('Полученный автомат распознает слово {}'.format(input_string))
else:
    print('Полученный автомат не распознает слово {}'.format(input_string))



Текущее состояние: 4
Полученный символ из введенной строки: a
Переход в состояние 1
Текущее состояние: 1
Полученный символ из введенной строки: b
Переход в состояние 2
Состояние автомата не конечное
Полученный автомат не распознает слово ab
